In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Exploratory Data analysis

In [ ]:
# importing necessary packages
from pandas_datareader import data
from fastai.imports import *
# from structured import add_datepart
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Loading into pandas dataframes

In [ ]:
## Memory optimization

# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type, categorical type
# Modified to add option to use float16

from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
train_feat_df = pd.read_csv('/kaggle/input/ashrae-energy-prediction/train.csv')
building_metd_df = pd.read_csv('/kaggle/input/ashrae-energy-prediction/building_metadata.csv')
weather_train_df = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_train.csv')
train_feat_df = reduce_mem_usage(train_feat_df, use_float16=True)
building_metd_df = reduce_mem_usage(building_metd_df, use_float16=True)
weather_train_df = reduce_mem_usage(weather_train_df, use_float16=True)

Examining the dataframes before joins

In [ ]:
print(train_feat_df.head(10))
print("shape: ", train_feat_df.shape)
print(train_feat_df.info())

In [ ]:
print(building_metd_df.head(10))
print("shape: ", building_metd_df.shape)
print(building_metd_df.info())

In [ ]:
print(weather_train_df.head(10))
print("shape: ", weather_train_df.shape)
print(weather_train_df.info())

Join train_feat_df {shape: (20216100, 4)} with building_metd_df {(1449, 6)} on foreign key building_id

In [ ]:
building_train_df = pd.merge(train_feat_df, building_metd_df, on='building_id')

Join building_train_df with weather data with join keys as site id and timestamp

In [ ]:
#checking object dtypes for datetime type
print(building_train_df['timestamp'].apply(type))
print(weather_train_df['timestamp'].apply(type))

Since str is the type of timestamp, it should be better if we converted this to a datetime type

In [ ]:
building_train_df['timestamp'] = pd.to_datetime(building_train_df['timestamp'])
weather_train_df['timestamp'] = pd.to_datetime(weather_train_df['timestamp'])

In [ ]:
# weather_building_df : wbt_df
wbt_df = pd.merge(building_train_df, weather_train_df,how='left', on=['site_id', 'timestamp'])

Explore the merged df

In [ ]:
print(building_train_df.head(10))
print("shape: ", building_train_df.shape)
print(building_train_df.info())

1.5+ GB memory usage - See how you can decrease the memory usage

In [ ]:
print(wbt_df.head(10))
print("shape: ", wbt_df.shape)
print(wbt_df.info())

We now have the following dataframes to perform eda with

1. train_feat_df
2. building_metd_df
3. weather_train_df
4. building_train_df
5. wbt_df

Check for duplicates

In [ ]:
List_Of_Df = [train_feat_df,building_metd_df,weather_train_df,building_train_df,wbt_df]

def duplicates(df):
    return df.duplicated().sum()

In [ ]:
# can use iter or for loop. use iter for faster processing
# d=iter(List_Of_Df)
# d2 = duplicates(next(d))
# print(d2)
for df in List_Of_Df:
    print(duplicates(df))

No duplicates found in any ofdataframes, seems to be a clean dataset

Find the Nan/nulls in our columns and rows 

In [ ]:
def missing_values(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [ ]:
for df in List_Of_Df:
    print(missing_values(df))

Fair to say that floor count, year_built, cloud coverage, precip_depth_1_hr are the major columns will null values. Reasonable estimations need to figures out for all 

for floor count: we can estimate the average floor count per site id based on floor counts of buildings in a given site id. or replace it by the average of the floor count throughout the dataset. 

Its hard to replace year built with estimations but it could be one hot encoded so all null year builts would fall into the same category

Cloud coverage will be the average per site for the specific day and so will the precipitation depth. 

Note: We will get a better estimation of how to fill up the nulls by looking at the respective distributions over the period of days, months and years

In [ ]:
#fromt the fastai/old packages 

def ifnone(a:Any,b:Any)->Any:
    "`a` if `a` is not None, otherwise `b`."
    return b if a is None else a

def make_date(df:DataFrame, date_field:str):
    "Make sure `df[field_name]` is of the right date type."
    field_dtype = df[date_field].dtype
    if isinstance(field_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        field_dtype = np.datetime64
    if not np.issubdtype(field_dtype, np.datetime64):
        df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)
        
def add_datepart(df:DataFrame, field_name:str, prefix:str=None, drop:bool=True, time:bool=False):
    "Helper function that adds columns relevant to a date in the column `field_name` of `df`."
    make_date(df, field_name)
    field = df[field_name]
    prefix = ifnone(prefix, re.sub('[Dd]ate$', '', field_name))
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start', 
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[prefix + n] = getattr(field.dt, n.lower())
    df[prefix + 'Elapsed'] = field.astype(np.int64) // 10 ** 9
    if drop: df.drop(field_name, axis=1, inplace=True)
    return df

In [ ]:
add_datepart(wbt_df, 'timestamp')

In [ ]:
wbt_df.head(10)

summarizing Numerical Variables

In [ ]:
wbt_df.describe(include = [np.number])

summarizing categorical variables

In [ ]:
wbt_df.describe(include = ['O'])

We will comeback to evaluate the describe later as for now it does not seem like a lot of outliers are present. 

lets visualize the target variable i.e. meter_reading

In [ ]:
def target_viz(target):
    plt.figure(figsize = (14, 6))
    plt.subplot(1,2,1)
    sns.boxplot(target)
    plt.subplot(1,2,2)
    sns.distplot(target, bins=20)
    plt.show()

In [ ]:
target_viz(wbt_df.meter_reading)

Highl skewed towards towards 0.

Identyfying potential outliers

In [ ]:
def outliers(df, target):
    stat = target.describe()
    print(stat)
    IQR = stat['75%'] - stat['25%']
    upper = stat['75%'] + 1.5 * IQR
    lower = stat['25%'] - 1.5 * IQR
    print('The upper and lower bounds for suspected outliers are {} and {}.'.format(upper, lower))
    lower_out = df[target < lower]
    print('Outliers below lower bound \n', lower_out)
    upper_out = df[target > upper]
    print('Outliers above upper bound \n', upper_out)

In [ ]:
outliers(wbt_df, wbt_df.meter_reading)

A high volume is above the upper bound. The upper bound is not the standard upper bound for this problem. The threshold for the upper bound has to be changed. How do we find out the upper bound threshold? 

Lets check for upper bound counts by type of meters as well as by primary use 

In [ ]:
2473655/20216100

about 12% {0: electricity, 1: chilledwater, 2: steam, 3: hotwater}

In [ ]:
wbt_df.loc[wbt_df.meter_reading > 642.5100000000001, 'meter'].value_counts()

steam meter is the highest with reading above upper bound, followed by chilledwater, electricity, hot water

In [ ]:
wbt_df.loc[wbt_df.meter_reading > 642.5100000000001, 'primary_use'].value_counts()

In [ ]:
wbt_df.loc[wbt_df.meter_reading > 642.5100000000001, 'site_id'].value_counts()

Continue from here tomorrow with more data exploration. but Just for fun lets run the h20 automl to get an idea of what the data is throwing at us

In [ ]:
# sns.barplot(x=wbt_df.meter_reading, y=wbt_df.primary_use)
# plt.show()
# plt.bar(x_pos, energy, color='green')
# plt.xlabel("Energy Source")
# plt.ylabel("Energy Output (GJ)")
# plt.title("Energy output from various fuel sources")

# plt.show()

In [ ]:
import h2o
from h2o.estimators import H2OXGBoostEstimator
from h2o.estimators.deeplearning import H2OAutoEncoderEstimator
from h2o.automl import H2OAutoML
h2o.init(nthreads = -1, max_mem_size = 8)

In [ ]:
df = h2o.H2OFrame(wbt_df)
y = "meter_reading"
splits = df.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
test = splits[1]

In [ ]:
aml = H2OAutoML(max_runtime_secs = 60, seed = 1234)
aml.train(y = y, training_frame = train, leaderboard_frame = test)

In [ ]:
aml.leaderboard.head()